# Hands on HDF5
This brief lab session gives an overview over the basic usage of the HDF5 file format for Python applications.

Work through the examples and try to change some of the settings...

### Sources:
* HDF5 documentation https://portal.hdfgroup.org/display/HDF5/HDF5
* h5py API: http://docs.h5py.org/en/stable/



## Creating a Data Set

In [1]:
import numpy as np
import h5py #this is the HDF5 lib 

In [2]:
#create some random data
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (10000,100))

In [3]:
# write it to the same file - in two different arrays
with h5py.File('hdf5_data.h5', 'w') as hdf: #note the write mode 'w'
    hdf.create_dataset('dataset1', data=matrix1)
    hdf.create_dataset('dataset2', data=matrix2)

## Reading 

In [4]:
#opening, listing and reading files
with h5py.File('hdf5_data.h5','r') as hdf:
    ls = list(hdf.keys())
    print('List of datasets in this file: \n', ls)
    data = hdf.get('dataset2') #here data is still some hdf5 object
    dataset1 = np.array(data) #need to convert it into numpy
    print('Shape of dataset1: \n', dataset1.shape)

List of datasets in this file: 
 ['dataset1', 'dataset2']
Shape of dataset1: 
 (10000, 100)


In [5]:
dataset1

array([[0.35298937, 0.19084537, 0.77813152, ..., 0.48492118, 0.9588813 ,
        0.89156765],
       [0.01259683, 0.80003331, 0.23631204, ..., 0.08398393, 0.12874762,
        0.97716024],
       [0.92925425, 0.78632283, 0.0049041 , ..., 0.23286537, 0.65432988,
        0.43653635],
       ...,
       [0.50616616, 0.40203264, 0.98987614, ..., 0.34526477, 0.20184567,
        0.82073561],
       [0.66550208, 0.68291915, 0.46589767, ..., 0.88377011, 0.81268262,
        0.52053982],
       [0.6857731 , 0.24782241, 0.25591154, ..., 0.76727604, 0.77452165,
        0.73217506]])

In [6]:
f = h5py.File('hdf5_data.h5', 'r')
ls = list(f.keys())
f.close()

In [7]:
ls

['dataset1', 'dataset2']

## Array Slicing
HDF5 support fancy array slicing - so we do not read all data just to get a slice: http://docs.h5py.org/en/latest/high/dataset.html#fancy-indexing

In [8]:
f = h5py.File('hdf5_data.h5', 'r')
f['dataset1'][100:120,:] # this notation mostly follows numpy notation -> try different slices!

array([[0.07922088, 0.20793175, 0.5460555 , ..., 0.75673413, 0.97837682,
        0.40633718],
       [0.48687628, 0.79027152, 0.52919646, ..., 0.77822124, 0.41939217,
        0.70902382],
       [0.40467815, 0.44258099, 0.83715752, ..., 0.80002386, 0.23864185,
        0.97292973],
       ...,
       [0.01935417, 0.32257056, 0.44610822, ..., 0.14527461, 0.5316757 ,
        0.73758511],
       [0.30711561, 0.16160028, 0.25174914, ..., 0.36860369, 0.25817175,
        0.23681105],
       [0.32005088, 0.75381222, 0.06145083, ..., 0.24929502, 0.41978205,
        0.68106082]])

## Creating Groups
We can organize data in groups, just like in file systems where we have files (here datasets) in folders (here groups) 

In [9]:
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (1000,1000))
matrix3 = np.random.random(size = (1000,1000))
matrix4 = np.random.random(size = (1000,1000))

In [10]:
with h5py.File('hdf5_groups.h5', 'w') as hdf:
    G1 = hdf.create_group('Group1')
    G1.create_dataset('dataset1', data = matrix1)
    G1.create_dataset('dataset4', data = matrix4)
 
    G21 = hdf.create_group('Group2/SubGroup1')
    G21.create_dataset('dataset3', data = matrix3)
    
    G22 = hdf.create_group('Group2/SubGroup2')
    G22.create_dataset('dataset2', data = matrix2)

## Reading Groups

In [11]:
with h5py.File('hdf5_groups.h5','r') as hdf:
    base_items = list(hdf.items())
    print('Items in the base directory:', base_items)
    G2 = hdf.get('Group2')
    G2_items = list(G2.items())
    print('Items in Group2:', G2_items)
    G21 = G2.get('/Group2/SubGroup1')
    G21_items = list(G21.items())
    print('Items in Group21:', G21_items)
    dataset3 = np.array(G21.get('dataset3'))
    print(dataset3.shape)


Items in the base directory: [('Group1', <HDF5 group "/Group1" (2 members)>), ('Group2', <HDF5 group "/Group2" (2 members)>)]
Items in Group2: [('SubGroup1', <HDF5 group "/Group2/SubGroup1" (1 members)>), ('SubGroup2', <HDF5 group "/Group2/SubGroup2" (1 members)>)]
Items in Group21: [('dataset3', <HDF5 dataset "dataset3": shape (1000, 1000), type "<f8">)]
(1000, 1000)


### What is happening? Interpret the results.

## Compress Data
HDF5 also support native data compression:

In [12]:
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (1000,1000))
matrix3 = np.random.random(size = (1000,1000))
matrix4 = np.random.random(size = (1000,1000))

In [13]:
with h5py.File('hdf5_groups_compressed.h5', 'w') as hdf:
    G1 = hdf.create_group('Group1')
    G1.create_dataset('dataset1', data = matrix1, compression="gzip", compression_opts=9)
    G1.create_dataset('dataset4', data = matrix4, compression="gzip", compression_opts=9)
 
    G21 = hdf.create_group('Group2/SubGroup1')
    G21.create_dataset('dataset3', data = matrix3, compression="gzip", compression_opts=9)
    
    G22 = hdf.create_group('Group2/SubGroup2')
    G22.create_dataset('dataset2', data = matrix2, compression="gzip", compression_opts=9)

## Attributes
We can add meta information in form of attributes of files, groups and datasets:

In [14]:
matrix1 = np.random.random(size = (1000,1000))
matrix2 = np.random.random(size = (10000,100))

In [15]:
# Create the HDF5 file
hdf = h5py.File('test.h5', 'w')

# Create the datasets
dataset1 = hdf.create_dataset('dataset1', data=matrix1)
dataset2 = hdf.create_dataset('dataset2', data=matrix2)

# Set attributes
dataset1.attrs['CLASS'] = 'DATA MATRIX'
dataset1.attrs['VERSION'] = '1.1'

hdf.close()

In [16]:
# Read the HDF5 file
hdf = h5py.File('test.h5', 'r')
ls = list(hdf.keys())
print('List of datasets in this file: \n', ls)
data = hdf.get('dataset1')
dataset1 = np.array(data)
print('Shape of dataset1: \n', dataset1.shape)
#read the attributes
k = list(data.attrs.keys())
v = list(data.attrs.values())
print(k[0])
print(v[0])
print(data.attrs[k[0]])

hdf.close()

List of datasets in this file: 
 ['dataset1', 'dataset2']
Shape of dataset1: 
 (1000, 1000)
CLASS
DATA MATRIX
DATA MATRIX


## HDF5 and ***Pandas***

In [17]:
import pandas as pd
# creates (or opens in append mode) an hdf5 file
hdf = pd.HDFStore('hdf5_pandas.h5')

In [18]:
df1 = pd.read_csv('../../DATA/FL_insurance_sample.csv')# put the dataset in the storage
hdf.put('DF1', df1, format='table', data_columns=True)

In [19]:
data = {
         "city": ["Tripoli", "Sydney", "Tripoli", "Rome", "Rome", "Tripoli","Rome", "Sydney", "Sydney"],
         "rank": ["1st", "2nd", "1st", "2nd", "1st", "2nd","1st", "2nd", "1st"], 
         "score1": [44, 48, 39, 41, 38, 44, 34, 54, 61],
         "score2": [67, 63, 55, 70, 64, 77, 45, 66, 72]
        }
        
df2 = pd.DataFrame(data, columns = ['city', 'rank','score1','score2'])

In [20]:
df2

,city,rank,score1,score2
0,Tripoli,1st,44,67
1,Sydney,2nd,48,63
2,Tripoli,1st,39,55
3,Rome,2nd,41,70
4,Rome,1st,38,64
5,Tripoli,2nd,44,77
6,Rome,1st,34,45
7,Sydney,2nd,54,66
8,Sydney,1st,61,72


In [21]:
hdf.put('DF2Key', df2,format='table', data_columns=True)

In [22]:
hdf.close() # close the hdf5 file

### now read

In [23]:
# open hdf5 file for reading
hdf = pd.HDFStore('hdf5_pandas.h5',mode='r')

In [24]:
hdf.keys()

['/DF1', '/DF2Key']

In [25]:
df1 = hdf.get('/DF1')

In [26]:
type(df1)

pandas.core.frame.DataFrame

In [27]:
df1.head()

,policyID,statecode,county,eq_site_limit,hu_site_limit,fl_site_limit,fr_site_limit,tiv_2011,tiv_2012,eq_site_deductible,hu_site_deductible,fl_site_deductible,fr_site_deductible,point_latitude,point_longitude,line,construction,point_granularity
0,119736,FL,CLAY COUNTY,498960.0,498960.00,498960.0,498960.0,498960.00,792148.90,0.0,9979.2,0.0,0,30.102261,-81.711777,Residential,Masonry,1
1,448094,FL,CLAY COUNTY,1322376.3,1322376.30,1322376.3,1322376.3,1322376.30,1438163.57,0.0,0.0,0.0,0,30.063936,-81.707664,Residential,Masonry,3
2,206893,FL,CLAY COUNTY,190724.4,190724.40,190724.4,190724.4,190724.40,192476.78,0.0,0.0,0.0,0,30.089579,-81.700455,Residential,Wood,1
3,333743,FL,CLAY COUNTY,0.0,79520.76,0.0,0.0,79520.76,86854.48,0.0,0.0,0.0,0,30.063236,-81.707703,Residential,Wood,3
4,172534,FL,CLAY COUNTY,0.0,254281.50,0.0,254281.5,254281.50,246144.49,0.0,0.0,0.0,0,30.060614,-81.702675,Residential,Wood,1


In [28]:
hdf.close()